In [1]:
import torch

In [2]:
#Just checking the torch version
print(torch.__version__)

1.0.0


### Activation Function

In [4]:
def activation(x):
    ''' Sigmoid Function
        Arguments => torch.tensor
        Used for probability coz its range is B/W 0 and 1'''
    return 1/(1 + torch.exp(x))

### Generating Data

In [9]:
torch.manual_seed(7)# Setting random seeds so things are predictable

#randn = Returns a tensor filled with random numbers from a normal distribution with mean `0` and variance `1` 
features = torch.randn((1,5)) # 1 row and 5 columns

#Weights again random normal variables
weights = torch.rand_like(features) # similar to randn but it takes another tensor as input

#Bias Term
bias = torch.randn((1,1))


Above I generated data we can use to get the output of our simple network. This is all just random for now, going forward we'll start using normal data. Going through each relevant line:

`features = torch.randn((1, 5))` creates a tensor with shape `(1, 5)`, one row and five columns, that contains values randomly distributed according to the normal distribution with a mean of zero and standard deviation of one. 

`weights = torch.randn_like(features)` creates another tensor with the same shape as `features`, again containing values from a normal distribution.

Finally, `bias = torch.randn((1, 1))` creates a single value from a normal distribution.

PyTorch tensors can be added, multiplied, subtracted, etc, just like Numpy arrays. In general, you'll use PyTorch tensors pretty much the same way you'd use Numpy arrays. They come with some nice benefits though such as GPU acceleration which we'll get to later

In [10]:
features , weights , bias

(tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]]),
 tensor([[0.2868, 0.2063, 0.4451, 0.3593, 0.7204]]),
 tensor([[-0.8948]]))

In [13]:
features.shape , weights.shape , bias.shape

(torch.Size([1, 5]), torch.Size([1, 5]), torch.Size([1, 1]))

# Exercise 1
> Calculate the output of the network with input features `features`, weights `weights`, and bias `bias`. Similar to Numpy, PyTorch has a [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum) function, as well as a `.sum()` method on tensors, for taking sums. Use the function `activation` defined above as the activation function

In [22]:
# y = activation-Function ( summation(wi*xi) + b )
summation = torch.sum(features * weights)
addingBias = summation + bias
output1 = activation(addingBias)

'''OR'''

output2 = activation((features * weights).sum() + bias)

In [23]:
output1 , output2

(tensor([[0.3860]]), tensor([[0.3860]]))

## We can also do this in 1shot with the help Matrix Multiplication
### torch.mm()  >>Prefer this
### OR torch.matmul() >> Here you may unexpected result || it gives the resut even if there is shape mismatch

In [21]:
torch.mm(features,weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/administrator/nightlies/pytorch-1.0.0/wheel_build_dirs/wheel_3.6/pytorch/aten/src/TH/generic/THTensorMath.cpp:940

### In matrix multiplication number of rows should be equal to number of columns of other matrix
## We can go any of the following

* `weights.reshape(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.
* `weights.resize_(a, b)` returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory. Here I should note that the underscore at the end of the method denotes that this method is performed **in-place**. Here is a great forum thread to [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)`
## Best to use => .view(a,b)

# Exercise 2
> Calculate the output of our little network using matrix multiplication.

In [24]:
output3 = activation(torch.mm(features , weights.view(5,1)) + bias)

In [25]:
output3

tensor([[0.3860]])

# Stacking

In [27]:
torch.manual_seed(7)

# Input features
features = torch.randn((1,3))

# Size of each layer
nInput = features.shape[1] #i.e 3
nHidden = 2 
nOutput = 1

# Weights
w1 = torch.randn(nInput , nHidden)

w2 = torch.randn(nHidden , nOutput)

#Bias Terms
b1 = torch.randn((1 , nHidden))

b2 = torch.randn((1 , nOutput))

# Exercise 3
> Calculate the output for this multi-layer network using the weights `W1` & `W2`, and the biases, `B1` & `B2`. 

In [28]:
features.shape , w1.shape , w2.shape , 

(torch.Size([1, 3]), torch.Size([3, 2]), torch.Size([2, 1]))

In [33]:
h = activation(torch.mm(features , w1) + b1)
output4 = activation(torch.mm(h , w2) + b2)

In [37]:
output4

tensor([[0.5290]])

## Numpy to Torch and back

Special bonus section! PyTorch has a great feature for converting between Numpy arrays and Torch tensors. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [38]:
import numpy as np

In [47]:
a = np.random.rand(4,3)
a

array([[0.96071436, 0.03170862, 0.10583322],
       [0.45539535, 0.22658808, 0.55613631],
       [0.67079288, 0.08615061, 0.66762049],
       [0.44350306, 0.61261427, 0.20748973]])

In [48]:
b = torch.from_numpy(a)
b

tensor([[0.9607, 0.0317, 0.1058],
        [0.4554, 0.2266, 0.5561],
        [0.6708, 0.0862, 0.6676],
        [0.4435, 0.6126, 0.2075]], dtype=torch.float64)

In [49]:
b.numpy()

array([[0.96071436, 0.03170862, 0.10583322],
       [0.45539535, 0.22658808, 0.55613631],
       [0.67079288, 0.08615061, 0.66762049],
       [0.44350306, 0.61261427, 0.20748973]])

* The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object,the other will change as well.
# _ is for inplace operations

In [51]:
b.mul_(2)

tensor([[1.9214, 0.0634, 0.2117],
        [0.9108, 0.4532, 1.1123],
        [1.3416, 0.1723, 1.3352],
        [0.8870, 1.2252, 0.4150]], dtype=torch.float64)

## Lets see if our numpy array has also changed or not

In [52]:
a

array([[1.92142871, 0.06341724, 0.21166644],
       [0.9107907 , 0.45317616, 1.11227261],
       [1.34158577, 0.17230122, 1.33524098],
       [0.88700611, 1.22522854, 0.41497946]])

# Cheers!